## 3.3 데이터 전처리

In [1]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
bts = pd.read_csv('data/bus_bts.csv')
jeju_life = pd.read_csv('data/jeju_financial_life_data.csv')
weather = pd.read_csv('data/weather.csv', encoding='cp949')
rain = pd.read_csv('data/rain.csv', encoding='utf-8')

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# 내부에 결과를 출력하도록 설정
%matplotlib inline

# 시각화 한글 폰트 설정
# window 사용자
# plt.rc('font', family='Malgun Gothic')
# macOS 사용자
plt.rc('font', family='AppleGothic')
# 마이너스 기호 출력
plt.rc('axes', unicode_minus=False)

# 분석에 문제가 없는 경고 메시지는 숨긴다.
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 학습 데이터와 테스트 데이터를 구분하기 위한 변수 생성
train['cue'] = 0
test['cue'] = 1

# 학습 데이터와 테스트 데이터 통합
df = pd.concat([train, test], axis=0)

### 3.3.1 내부 데이터를 통한 변수 생성

#### 3.3.1.1 탐색적 데이터 분석을 통한 변수

* 요일을 나타내는 변수(weekday)

In [4]:
# datetime 변수형으로 변환
df['date'] = pd.to_datetime(df['date'])

# 요일 추출 (0-월요일~6-일요일)
df['weekday'] = df['date'].dt.weekday

In [5]:
df[['weekday']].head()

,weekday
0,6
1,6
2,6
3,6
4,6


* 요일별 평균 탑승객 수

In [6]:
# 요일별 평균 탑승 승객수를 구하는 함수
def week_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)

    # 일괄적으로 1의 값을 가지는 'weekdaymean'변수 생성
    df['weekdaymean'] = 1

    # 각 요일에 해당하는 인덱스 추출
    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    # 인덱스를 활용하여 'weekdaymean'의 값을 각 요일에 맞는 평균 탑승 승객수로 변경
    df.iloc[index0,-1] = train_data.query('weekday==0')['18~20_ride'].mean()
    df.iloc[index1,-1] = train_data.query('weekday==1')['18~20_ride'].mean()
    df.iloc[index2,-1] = train_data.query('weekday==2')['18~20_ride'].mean()
    df.iloc[index3,-1] = train_data.query('weekday==3')['18~20_ride'].mean()
    df.iloc[index4,-1] = train_data.query('weekday==4')['18~20_ride'].mean()
    df.iloc[index5,-1] = train_data.query('weekday==5')['18~20_ride'].mean()
    df.iloc[index6,-1] = train_data.query('weekday==6')['18~20_ride'].mean()

    return df

# 함수를 실행하여 변수 생성
df = week_mean()

In [7]:
df[['weekdaymean']].head()

,weekdaymean
0,1.034282
1,1.034282
2,1.034282
3,1.034282
4,1.034282


* 버스 종류별 평균 탑승객 수

In [8]:
# 버스 종류별 평균 탑승 승객수를 구하는 함수
def inout_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)

    # 일괄적으로 1의 값을 가지는 in_out_mean 변수 생성
    df['in_out_mean'] = 1

    # 버스 종류별 인덱스 추출
    in_index = df.query('in_out == "시내"').index # 시내버스
    out_index = df.query('in_out == "시외"').index # 시외버스

    # 인덱스를 활용하여 in_out_mean의 값을 각 버스 종류에 맞는
    # 평균 탑승 승객수로 변경
    df.iloc[in_index,-1] = train_data.query('in_out == "시내"')['18~20_ride'].mean()
    df.iloc[out_index,-1] = train_data.query('in_out == "시외"')['18~20_ride'].mean()

    return df

# 함수를 실행하여 변수 생성
df = inout_mean()

In [9]:
df[['in_out_mean']].head()

,in_out_mean
0,2.044345
1,2.044345
2,2.044345
3,1.228499
4,1.228499


* 일별 오전 시간대의 총 탑승객 수

In [10]:
# 날짜별 오전 시간에 탑승한 총 탑승객 수
f = df.groupby('date')['6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride'].sum().reset_index()

# 변수명 변경
f.columns = ['date', '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum', '9~10_ride_sum', '10~11_ride_sum']

# 기존 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, f, how='left', on='date')

In [11]:
df[['date', '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum', '9~10_ride_sum', '10~11_ride_sum']].head()

,date,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum
0,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
1,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
2,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
3,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
4,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0


#### 3.3.1.2 도메인 조사를 통한 변수

* 배차 간격

In [12]:
# 탑승 날짜를 datetime 형으로 변환
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

# 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호 기준으로 정렬
f = bts.sort_values(by=['geton_date', 'geton_station_code', 'bus_route_id', 'geton_time2'], ascending=True).reset_index()

In [13]:
from tqdm import notebook

# 시차를 넣기 위한 빈 리스트 생성
interval_list = []
for i in notebook.tqdm(range(0, len(f)-1)):
  # 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호가 같다면
  if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) & (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) & (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):
    # 탑승 날짜의 차를 interval_list에 넣기
    interval_list.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)

  else:
    # 같지 않다면 0을 interval_list에 넣기
    interval_list.append(0)

interval_list.insert(0, '0')

  0%|          | 0/2409413 [00:00<?, ?it/s]

In [14]:
f['interval'] = interval_list         # interval_list를 interval 변수로 생성
# interval 변수의 값을 문자형으로 변환 후 시:분:초 형태로 변환
f['interval'] = f['interval'].astype(str).str[7:].replace('', '00:00:00')
f['bus_route_id'] = f['bus_route_id'].astype(object)

In [30]:
def get_min(time_str):
    h, m, s = time_str.split(':')
    return (int(h) * 3600 + int(m) * 60 + int(s)) / 60
# interval 함수의 모든 값에 대해 함수 적용하여 나온 값을 time 리스트에 넣기
time = []
for i in f['interval']:
    time.append(get_min(i))

f['time'] = time       # time 리스트를 time 변수로 생성
f['time'] = f['time'].astype(int)       # time 변수를 int 타입으로 변경
f = f[f['time'] > 3]                    # 간격이 3분보다 작은 것 제외
f = f[f['time'] < 180]                  # 간격이 3시간보다 큰 것 제외

In [31]:
# 버스 노선별 버스 간격의 평균 bus_interval 변수 생성
data = f.groupby('bus_route_id')['time'].agg([('bus_interval', 'mean')]).reset_index()
data['bus_interval'] = data['bus_interval'].astype(int)

# 변수 생서에 많은 시간이 소요되므로 csv 파일로 저장
data.to_csv('bus_interval.csv', index=False)

In [32]:
# csv 파일이 성공적으로 저장됐다면 다음 과정부터 진행
import numpy as np
df['bus_route_id'] = df['bus_route_id'].astype(np.int64)
bus_interval = pd.read_csv('bus_interval.csv')

# 기종의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, bus_interval, how='left', on='bus_route_id')
df['bus_interval'] = df['bus_interval'].fillna(9999)     # 병합 중 생기는 NA 값 처리

In [33]:
df[['bus_interval']].head()

,bus_interval
0,34.0
1,34.0
2,34.0
3,34.0
4,34.0
